# Final List Converter

In [47]:
import pandas as pd
import numpy as np

In [374]:
def load_format(filename, header=5, sheet_name="AE"):
    df = pd.read_excel(filename, header=5, sheet_name="AE")
    
    colnames = ["이상사례명(MedDRA_SOC_ENG)","이상사례명(MedDRA_SOC_KOR)",
            "이상사례명(MedDRA_PT_ENG)","이상사례명(MedDRA_PT_KOR)",
            "Expectedness","차수","중대성","ADR 여부", "자료원"]
    
    df = df[colnames]
    
    df["SOC"] = df["이상사례명(MedDRA_SOC_ENG)"] + " (" + df["이상사례명(MedDRA_SOC_KOR)"] + ")"
        
    df["PT"] = df["이상사례명(MedDRA_PT_ENG)"] + " (" + df["이상사례명(MedDRA_PT_KOR)"] + ")"
    df = df.drop(columns=["이상사례명(MedDRA_SOC_ENG)",
                              "이상사례명(MedDRA_SOC_KOR)",
                              "이상사례명(MedDRA_PT_ENG)","이상사례명(MedDRA_PT_KOR)"])
    
    
       
    return df


In [375]:
def process_values(df):
    binary_cols = ["Expectedness","중대성", "ADR 여부", "자료원"]
    edited = list()
    idx = 0
    while idx < len(binary_cols):
        if len(pd.unique(df[binary_cols[idx]])) > 2:
            print("Column '{}' is not binary: ".format(binary_cols[idx]), end="")
            print(pd.unique(df[binary_cols[idx]]))

            choice = input("\tSelect the following option:\n\t1) Remove rows with specified characters (제외할 경우) \n\t2) Replace specific characters (오타인 경우) \n\t3) Correct. Next \n\tNumber: ")
            
            if choice == "1":
                edited.append(idx)
                problem_rows = input("\t\tRemove rows with (exact) match:" )
                df = df.loc[~df[binary_cols[idx]].str.contains(problem_rows)]

            elif choice == "2":
                edited.append(idx)
                remove = input("\tString to remove (e.g., \\n): ")
                value = input("\tValue to replace: ")
                df[binary_cols[idx]] = df[binary_cols[idx]].str.replace(remove, value)
                
            elif choice == "3":
                idx += 1
                continue
                
            else:
                print("Enter 1, 2 or 3")
                idx -= 1
        idx += 1
            
    for i in edited:
        assert len(pd.unique(df[binary_cols[i]])) <= 2
        
    print("*"*40)
    print("Summary")
    for binary in binary_cols:
        print("{} : {}".format(binary, pd.unique(df[binary])))
    print("*"*40)
    
    return df

In [376]:
def process_time(df):
    choice = ""
    while (choice != "3"):
        print("Check if 차수 is correct:\n")
    
        print(pd.unique(df["차수"]), end='\n\n')
        choice = input("\tSelect the following option:\n\t1) Remove rows with specified characters (제외할 경우) \n\t2) Replace specific characters (오타인 경우) \n\t3) Correct. Next \n\tNumber: ")
        if choice == "1":
            problem_rows = input("\t\tRemove rows with (exact) match:" )

            print("*"*40)
            print("Removing following rows...\n")
            print("{}\n\n".format(df.loc[df["차수"].str.contains(problem_rows)]))
            print("*"*40)

            df = df.loc[~df["차수"].str.contains(problem_rows)]

        elif choice == "2":
            remove = input("\tCharacter(s) to replace (e.g., \\n): ")
            value = input("\tReplacement character(s): ")
            df["차수"] = df["차수"].str.replace(remove, value)
            
        elif choice == "3":
            continue
            
        else:
            print("Enter 1, 2 or 3")

    
    return df

In [421]:
# filename = "Lynparza_PMS_2-1차_Spontaneous AE Coding Confirmation_V1.0_20210527의 복사본.xlsx"
filename = "Lynparza_PMS_2-2차_Spontaneous AE Coding Confirmation_V1.0_final.xlsx"

# load data
data = load_format(filename)

# process variable values
data_binary = process_values(data)
data_processed = process_time(data_binary)

Column 'Expectedness' is not binary: ['Unexpected AE' 'Expected AE' nan ' Unexpected AE']


	Select the following option:
	1) Remove rows with specified characters (제외할 경우) 
	2) Replace specific characters (오타인 경우) 
	3) Correct. Next 
	Number:  3


Column '중대성' is not binary: ['예' '아니오' '이니오']


	Select the following option:
	1) Remove rows with specified characters (제외할 경우) 
	2) Replace specific characters (오타인 경우) 
	3) Correct. Next 
	Number:  2
	String to remove (e.g., \n):  이
	Value to replace:  아


****************************************
Summary
Expectedness : ['Unexpected AE' 'Expected AE' nan ' Unexpected AE']
중대성 : ['예' '아니오']
ADR 여부 : ['ADR' 'non ADR']
자료원 : ['자발보고' '조사연구']
****************************************
Check if 차수 is correct:

['2-2차' '2-2차 ->2-1차' '2-2차 -> 2-1차' '2-2차 -> 1-2차' '2-2차 -> 1-1차']



...

	Select the following option:
	1) Remove rows with specified characters (제외할 경우) 
	2) Replace specific characters (오타인 경우) 
	3) Correct. Next 
	Number:  1
		Remove rows with (exact) match: 2-2차 -> 1-2차


****************************************
Removing following rows...

      Expectedness            차수  중대성   ADR 여부   자료원  \
34     Expected AE  2-2차 -> 1-2차  아니오      ADR  조사연구   
35     Expected AE  2-2차 -> 1-2차  아니오      ADR  조사연구   
36   Unexpected AE  2-2차 -> 1-2차    예  non ADR  조사연구   
37   Unexpected AE  2-2차 -> 1-2차  아니오  non ADR  조사연구   
38     Expected AE  2-2차 -> 1-2차  아니오  non ADR  조사연구   
39   Unexpected AE  2-2차 -> 1-2차  아니오  non ADR  조사연구   
59   Unexpected AE  2-2차 -> 1-2차    예  non ADR  조사연구   
66   Unexpected AE  2-2차 -> 1-2차    예      ADR  자발보고   

                                                  SOC  \
34  Gastrointestinal disorders (각종 위장관 장애         ...   
35  Metabolism and nutrition disorders (대사 및 영양 장애...   
36  Neoplasms benign, malignant and unspecified (i...   
37  Nervous system disorders (각종 신경계 장애           ...   
38  Gastrointestinal disorders (각종 위장관 장애         ...   
39  Respiratory, thoracic and mediastinal disorder...   
59  Neoplasms benign, malig

	Select the following option:
	1) Remove rows with specified characters (제외할 경우) 
	2) Replace specific characters (오타인 경우) 
	3) Correct. Next 
	Number:  1
		Remove rows with (exact) match: 2-2차 -> 1-1차


****************************************
Removing following rows...

   Expectedness            차수  중대성 ADR 여부   자료원  \
41  Expected AE  2-2차 -> 1-1차    예    ADR  조사연구   
42  Expected AE  2-2차 -> 1-1차    예    ADR  조사연구   
43  Expected AE  2-2차 -> 1-1차  아니오    ADR  조사연구   
44  Expected AE  2-2차 -> 1-1차  아니오    ADR  조사연구   
92  Expected AE  2-2차 -> 1-1차    예    ADR  조사연구   

                                                  SOC  \
41  Blood and lymphatic system disorders (혈액 및 림프계...   
42  Blood and lymphatic system disorders (혈액 및 림프계...   
43  Blood and lymphatic system disorders (혈액 및 림프계...   
44  Gastrointestinal disorders (각종 위장관 장애         ...   
92  Blood and lymphatic system disorders (혈액 및 림프계...   

                            PT  
41       Neutropenia (중성구 감소증)  
42  Thrombocytopenia (혈소판 감소증)  
43                Anaemia (빈혈)  
44               Vomiting (구토)  
92                Anaemia (빈혈)  


****************************************
Check if 차수 is correct:

['2-2차']



	Select the following option:
	1) Remove rows with specified characters (제외할 경우) 
	2) Replace specific characters (오타인 경우) 
	3) Correct. Next 
	Number:  3


In [422]:
def identify_adr(df):
    '''
    Identifies ADF여부 values (e.g., \"non-ADR\" vs. \"non ADR\")
    '''
    adr_stat = pd.unique(df["ADR 여부"])
    # adr_stat = ["ADR positive", "ADR negative"]

    found = False
    non_idx = -1
    for i in range(2):
        if "non" in adr_stat[i]:
            found = True
            non_idx = i

    if not found:
        print("'ADR 여부' column에서 'ADR', 'non-ADR'을 찾지 못함:")
        print("현재 ADR 여부 column:")
        print(adr_stat)
        print("직접 ADR을 선택해주세요")
        truth = input("'{}'이 ADR 맞을까요? (혹은 '{}'이 non-ADR). Type y/n".format(adr_stat[0], adr_stat[1]))
        if truth:
            non_idx = 1
        else:
            non_idx = 0
    return non_idx

In [423]:
def make_expectedness_key(df):
    '''
    Works with data_processed     
    '''
    df["SOC-PT"] = df["SOC"] + "-" + df["PT"]
    key = df.drop(columns=['차수', '중대성', 'ADR 여부', '자료원', 'SOC', 'PT']).reset_index(drop=True)
    key = key.drop_duplicates("SOC-PT")
    key_cols = key['SOC-PT']
    key = key.T
    key.columns = key_cols
    key = key.reset_index(drop=True)
    key = key.drop(index=1)
    return key

In [450]:
def make_medDRA_key(df):
    key = df[["SOC-PT original", "SOC-PT"]]
    key = key.drop_duplicates("SOC-PT")
    key_cols = key['SOC-PT']
    key = key.T
    key.columns = key_cols
    key = key.reset_index(drop=True)
    key = key.drop(index=1)
    return key

In [494]:
def transform_format(data_processed_in, mode=0):
    '''
    mode = 0 : SOC and PT 합치 된 포맷 (합계 포함)
    mode = 1 : SOC and PT 분리 된 포맷 (default)
    '''
    data_processed_in = data_processed_in.copy()
    data_processed_in["SOC-PT original"] = data_processed_in["SOC"] + "-" + data_processed_in["PT"]
    data_processed_in["SOC"] = data_processed_in["SOC"].apply(lambda x : "".join(x.split()))
    data_processed_in["PT"] = data_processed_in["PT"].apply(lambda x : "".join(x.split())) 
    data_processed_in["SOC-PT"] = data_processed_in["SOC"] + "-" + data_processed_in["PT"]
    
    m_key = make_medDRA_key(data_processed_in)

    data_processed_in = data_processed_in.sort_values(["SOC", "PT"])
    table = data_processed_in.groupby(["SOC", "PT", "ADR 여부",
                                "차수", "중대성", "자료원"]).count().unstack(level=-2,
                                                                     fill_value=0).unstack(level=-2,
                                                                                           fill_value=0).unstack(level=-2,
                                                                                                                 fill_value=0)["Expectedness"]
    

    # Since non-ADR (i.e., AE) includes ADR events too
    # First, idenify ADR value
    non_adr = identify_adr(data_processed_in)
    if non_adr == 1:
        adr = 0
    else:
        adr = 1
        
    adr_status = pd.unique(data_processed_in["ADR 여부"])
    seriousness = pd.unique(data_processed_in["중대성"])
    time = pd.unique(data_processed_in["차수"])
    for s in seriousness:
        for t in time:
            table[s, t, adr_status[non_adr]] = np.array(table[s, t, adr_status[non_adr]]) + np.array(table[s, t, adr_status[adr]])
    
    
    # Add expectedness column
    table_df = table.reset_index()
    table_df['SOC-PT'] = table_df['SOC'] + '-' + table_df['PT']
    
    e_key = make_expectedness_key(data_processed_in)
    table["Expectedness"] = e_key[table_df["SOC-PT"]].values[0]    
    
    if mode == 1:
        return table
    
    
    # Merge PT and SOC into one column
    table_df = table.reset_index()
    table_df['SOC-PT'] = table_df['SOC'] + '-' + table_df['PT']
    table_df['SOC-PT original'] = m_key[table_df["SOC-PT"]].values[0]
    table_df[['SOC', 'PT']] = table_df['SOC-PT original'].str.split('-', 1, expand=True)
    table_df["stat"] = 1
    sum_stat = table_df.groupby("SOC").sum()
    sum_stat["Expectedness"] = ""
    sum_stat["자료원"] = ""
    sum_stat["stat"] = 0
    sum_stat = sum_stat.reset_index()
    sum_stat["PT"] = sum_stat["SOC"]
    
    combined = pd.concat([table_df, sum_stat]).reset_index(drop=True)
    combined = combined.sort_values(["SOC", "stat"]).reset_index(drop=True)
    combined = combined.drop(columns=["SOC", "stat", "SOC-PT", "SOC-PT original"])    
    return combined

In [498]:
final = transform_format(data_processed, 1)

In [499]:
final

중대성                                                                                                         아니오  \
차수                                                                                                         2-2차   
ADR 여부                                                                                                      ADR   
SOC                                                PT                                                 자료원         
Bloodandlymphaticsystemdisorders(혈액및림프계장애)         Anaemia(빈혈)                                        자발보고    0   
                                                                                                      조사연구    0   
                                                   Myelosuppression(골수억제)                             자발보고    0   
Cardiacdisorders(각종심장장애)                           Cardiacfailure(심부전)                                조사연구    0   
Gastrointestinaldisorders(각종위장관장애)                 Abdominalhernia(복부탈장)                              조사연구    0   
                                                   Abdominalpain(복통)                                  조사연구    0   
                                                   Dyspepsia(소화불량)                                    조사연구    0   
                                                   Nausea(구역)                                         자발보고    3   
                                                                                                      조사연구    1   
                                                   Vomiting(구토)                                       자발보고    2   
                                                                                                      조사연구    1   
Generaldisordersandadministrationsiteconditions... Fatigue(피로)                                        조사연구    3   
                                                   Pyrexia(발열)                                        조사연구    0   
Hepatobiliarydisorders(간담도장애)                      Cholangitis(담관염)                                   조사연구    0   
Infectionsandinfestations(감염및기생충감염)                Abscess(농양)                                        조사연구    0   
                                                   Cellulitis(연조직염)                                   조사연구    0   
                                                   Periodontitis(치주염)                                 조사연구    0   
                                                   Urinarytractinfection(요로감염)                        조사연구    0   
Investigations(임상검사)                               Aspartateaminotransferaseincreased(아스파르트산아미노전이효... 조사연구    0   
                                                   Bloodcreatinineincreased(혈액크레아티닌증가)                조사연구    0   
                                                   Haemoglobindecreased(헤모글로빈감소)                      자발보고    0   
Metabolismandnutritiondisorders(대사및영양장애)           Decreasedappetite(식욕감소)                            자발보고    1   
                                                   Feedingdisorder(섭식장애)                              자발보고    1   
Musculoskeletalandconnectivetissuedisorders(근골격... Muscularweakness(근육쇠약)                             조사연구    0   
Neoplasmsbenign,malignantandunspecified(inclcys... Malignantneoplasmprogression(악성신생물진행)              조사연구    0   
                                                   Neoplasmmalignant(악성신생물)                           조사연구    0   
Nervoussystemdisorders(각종신경계장애)                    Dysarthria(조음장애)                                   조사연구    0   
                                                   Headache(두통)                                       조사연구    0   
Renalandurinarydisorders(신장및요로장애)                  Acutekidneyinjury(급성신손상)                           조사연구    0   
                                                   Hydronephrosis(수신증)                                조사연구    0   
                                                   Renalimpairment(신기능장애)                 

In [501]:
final.to_excel("Lynparza_PMS_2-2차_일람표 test_sep2.xlsx")

In [509]:
og = load_format(filename)

In [520]:
og.loc[og["PT"].str.contains("neoplasm")]

,Expectedness,차수,중대성,ADR 여부,자료원,SOC,PT
25,Unexpected AE,2-2차,예,non ADR,조사연구,"Neoplasms benign, malignant and unspecified (i...",Malignant neoplasm progression (악성 신생물 진행)
26,Unexpected AE,2-2차,예,ADR,조사연구,"Neoplasms benign, malignant and unspecified (i...",Malignant neoplasm progression (악성 신생물 진행)
31,Unexpected AE,2-2차 -> 2-1차,예,non ADR,조사연구,"Neoplasms benign, malignant and unspecified (i...",Malignant neoplasm progression (악성 신생물 진행)
36,Unexpected AE,2-2차 -> 1-2차,예,non ADR,조사연구,"Neoplasms benign, malignant and unspecified (i...",Malignant neoplasm progression (악성 신생물 진행)
59,Unexpected AE,2-2차 -> 1-2차,예,non ADR,조사연구,"Neoplasms benign, malignant and unspecified (i...",Malignant neoplasm progression (악성 신생물 진행)


In [521]:
og.iloc[68]

Expectedness                                        Unexpected AE
차수                                                           2-2차
중대성                                                             예
ADR 여부                                                    non ADR
자료원                                                          조사연구
SOC             Neoplasms benign, malignant and unspecified (i...
PT                                    Neoplasm malignant (악성 신생물)
Name: 68, dtype: object

In [525]:
og.loc[og["SOC"].str.contains("Neoplasms benign, malignant and unspecified")]

,Expectedness,차수,중대성,ADR 여부,자료원,SOC,PT
25,Unexpected AE,2-2차,예,non ADR,조사연구,"Neoplasms benign, malignant and unspecified (i...",Malignant neoplasm progression (악성 신생물 진행)
26,Unexpected AE,2-2차,예,ADR,조사연구,"Neoplasms benign, malignant and unspecified (i...",Malignant neoplasm progression (악성 신생물 진행)
28,Unexpected AE,2-2차 -> 2-1차,예,non ADR,조사연구,"Neoplasms benign, malignant and unspecified (i...",Metastases to peritoneum (복막에 전이)
31,Unexpected AE,2-2차 -> 2-1차,예,non ADR,조사연구,"Neoplasms benign, malignant and unspecified (i...",Malignant neoplasm progression (악성 신생물 진행)
33,Unexpected AE,2-2차 -> 2-1차,예,non ADR,조사연구,"Neoplasms benign, malignant and unspecified (i...",Metastases to meninges (수막에 전이)
36,Unexpected AE,2-2차 -> 1-2차,예,non ADR,조사연구,"Neoplasms benign, malignant and unspecified (i...",Malignant neoplasm progression (악성 신생물 진행)
59,Unexpected AE,2-2차 -> 1-2차,예,non ADR,조사연구,"Neoplasms benign, malignant and unspecified (i...",Malignant neoplasm progression (악성 신생물 진행)
66,Unexpected AE,2-2차 -> 1-2차,예,ADR,자발보고,"Neoplasms benign, malignant and unspecified (i...",Myelodysplastic syndrome (골수 형성 이상 증후군)
68,Unexpected AE,2-2차,예,non ADR,조사연구,"Neoplasms benign, malignant and unspecified (i...",Neoplasm malignant (악성 신생물)
